<a href="https://colab.research.google.com/github/vannguyenhoai43-hash/ecommerce-category-growth-analysis/blob/main/notebooks/05_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**PHẦN 5: XU HƯỚNG TĂNG TRƯỞNG**

Phân tích xu hướng trong các tháng gần nhất nhằm đánh giá động lực tăng/giảm của ngành hàng và sản phẩm, thay vì chỉ nhìn vào biến động tại một thời điểm. Từ đó đẩy mạnh ngành hàng, sản phẩm tiềm năng

**5.1. Xu hướng theo ngành hàng level 2**

Đối với từng LV2, theo dõi biến động diff_ado và diff_gmv theo tháng để xác định:

- Ngành hàng có xu hướng tăng trưởng liên tục

- Ngành hàng có xu hướng suy giảm liên tục

**Triển khai**:
 - B1: Lọc bỏ tháng ko có prev ( tháng đầu dữ liệu)
 - B2: Tạo bảng pivot ( index = level 2, columns = year_month, values = diff)
 - B3: Lấy các ngành hàng level 2 có từ 2 tháng gần nhất trở lên > 0 hoặc < 0

**Mục tiêu** là xác định nhóm ngành đang hình thành động lực tăng trưởng bền vững hoặc có dấu hiệu suy giảm kéo dài.

**5.2. Xu hướng theo nhóm sản phẩm**

Ở cấp độ sản phẩm, tiến hành phân lớp theo mức độ tăng trưởng/suy giảm nhằm bắt kịp xu hướng thị trường, đẩy mạnh nhóm sản phẩm vào mùa.

Sản phẩm được phân tích theo 3 lớp:

**Lớp 1: Sản phẩm có đóng góp lớn**

    Điều kiện:

    max(share_ado) thuộc nhóm cao nhất (top-down)

*Mục đích: Lấy tỷ trọng lớn nhất của items trong các tháng ( ví dụ Tháng 7,8,9 tỷ trọng lần lượt của items là 3%,10%,6% thì tỷ trọng lớn nhất tại tháng 8 - ứng với 10%). Cách này để ko bỏ sót các sản phẩm đã từng có tỷ trọng cao trong các tháng trước.*

**Lớp 2: Tăng trưởng/ Suy giảm liên tiếp**

    Điều kiện:

    diff_ado > 0 với tăng trưởng và < 0 với suy giảm trong ít nhất 2 tháng liên tiếp gần nhất

*Mục đích: lọc các sản phẩm có xu hướng tăng giảm rõ ràng*

**Lớp 3: Tăng trưởng/ Suy giảm với quy mô lớn**

    Điều kiện:

    abs(sum(diff_ado)) trong các tháng ở mức cao (top-down)

*Mục đích: lọc các sản phẩm có biến động đủ lớn về mặt quy mô.*

=> Thỏa mãn cùng lúc 3 điều kiện này => lọc đc nhóm sản phẩm tiêu biểu có tính trending.

**Triển khai**

B1: Tạo hai bảng sau:

    Bảng tăng trưởng (diff_ado, diff_gmv) => pivot (index = 'keywords',columns='year_month', values = diff_col, aggfunc='sum').

    Bảng tỷ trọng của từng nhóm items => pivot (index = ['keywords','level2_kpi_category','level1_kpi_category'],columns='year_month',values = share_col, aggfunc='sum')

B2: Lấy tỷ trọng lớn nhất của items trong các tháng

B3: Gộp hai bảng này lại

B4: Tính tốc độ tăng trưởng và phân vị 70% để lập điều kiện

    increase_score = abs(sum(diff_ado các tháng))
    q_share = (tỷ trọng cao nhất của items).quantile(0.7)
    q_increase = increase_score .quantile(0.7)

B5: Điều kiện lọc sản phẩm:

  Xu hướng giảm:

    Tỷ trọng lớn nhất của items > q_share,
    Tăng trưởng các tháng < 0,
    increase_score > q_increase

  Xu hướng tăng:
  
    Tỷ trọng lớn nhất của items > q_share,
    Tăng trưởng các tháng > 0,
    increase_score > q_increase

**Ý nghĩa phân tích**

- Nhận diện xu hướng tăng/giảm mang tính hệ thống.

- Giúp loại bỏ nhiễu ngắn hạn và phản ánh xu hướng thực tế hơn.

##Import Data

In [ ]:
# chạy lại repo
%cd /content
!rm -rf ecommerce-category-growth-analysis

/content


In [ ]:
!git clone -q https://github.com/vannguyenhoai43-hash/ecommerce-category-growth-analysis.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append("/content/ecommerce-category-growth-analysis/src")

from load_data import load_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# DF theo level 2
df_source_lv2= load_data(month='2024-09')['source_lv2']

In [ ]:
# DF theo items
df_source_items= load_data(month='2024-09')['source_items']

In [ ]:
# DF theo items tháng hiện tại
df_cur_items= load_data(month='2024-09')['cur_items']

##Xu hướng theo ngành hàng

In [ ]:
# Loại các tháng ko có data last month (tháng đầu dữ liệu)
df_source_lv2_2 = df_source_lv2.copy()
df_lv2_valid = df_source_lv2_2.dropna(subset=['ADO_M_1'])

In [ ]:
# Tạo bảng tăng trưởng ( tính diff_ado, diff_gmv)
from metrics import growth_by_mul_level
df_lv2_gr_month= growth_by_mul_level(df_lv2_valid, ['year_month','level1_kpi_category','level2_kpi_category'])

In [ ]:
# Insight về xu hướng ngành hàng tăng
from metrics import trend_grow
insight_01 = trend_grow(df_lv2_gr_month,"Vehicle Essentials")
insight_02 = trend_grow(df_lv2_gr_month,"Home & Technical Supplies")

In [ ]:
# Insight về xu hướng ngành hàng giảm
from metrics import trend_down
insight_03 = trend_down(df_lv2_gr_month,"Vehicle Essentials")
insight_04 = trend_down(df_lv2_gr_month,"Home & Technical Supplies")

## Xu hướng theo sản phẩm

### Xu hướng giảm

In [ ]:
# Bảng items nhiều tháng
from metrics import clean_and_shorten
items_source = df_source_items.copy()
items_source['keywords']= items_source['item_name'].apply(clean_and_shorten)

In [ ]:
item_df = df_cur_items.copy()
item_df ['keywords']= item_df['item_name'].apply(clean_and_shorten)

In [ ]:
# list items ado sau lọc
from metrics import items_keep
items_keep_ado = items_keep(item_df,'ado')

In [ ]:
# DF items ado 3 tháng
items_ado_month = items_source.loc[items_source['keywords'].isin(items_keep_ado)].copy()

In [ ]:
# bảng diff ado
df_items_diff_ado = growth_by_mul_level(items_ado_month, ['year_month','keywords'])

In [ ]:
# Bảng Total share theo ado
from metrics import total_share_items
df_items_share_ado = total_share_items(items_ado_month,'ado')

In [ ]:
# Sản phẩm có xu hướng giảm ado
from metrics import items_trend_down
items_trend_down_ado = items_trend_down(df_items_diff= df_items_diff_ado, df_items_share= df_items_share_ado ,metric='ado')
items_trend_down_ado

,level1_kpi_category,level2_kpi_category,keywords,share_max_3m,decline_score
33257,Home & Technical Supplies,Construction Materials,xe kéo đi chợ thị,0.009436,7.872043
19091,Home & Technical Supplies,Construction Materials,lưới nắng 90 may đóng,0.011134,6.963755
36566,Home & Technical Supplies,Construction Materials,đầu vòi bồn chén tăng,0.007044,6.459334
3245,Home & Technical Supplies,Construction Materials,bạt nắng mưa trời đa,0.010455,5.641129
33259,Home & Technical Supplies,Construction Materials,xe kéo đi thị chợ,0.005570,4.143010
...,...,...,...,...,...
16079,Vehicle Essentials,Vehicle Maintenance,kem autosol metal polish dùng,0.023147,23.518357
32753,Vehicle Essentials,Vehicle Maintenance,vết ố kính vế bẩn,0.010219,13.584864
6821,Vehicle Essentials,Vehicle Maintenance,cana mờ vết xước,0.007642,10.587146
17132,Vehicle Essentials,Vehicle Maintenance,khăn lau 2 mặt ô,0.022558,8.294034


### Xu hướng tăng

In [ ]:
# list items ado sau lọc
from metrics import items_keep_grow
items_keep_ado_grow = items_keep_grow(item_df,'ado')

In [ ]:
# DF items ado 3 tháng
items_ado_grow_month = items_source.loc[items_source['keywords'].isin(items_keep_ado_grow)].copy()

In [ ]:
# bảng diff ado
df_items_grow_diff_ado = growth_by_mul_level(items_ado_grow_month, ['year_month','keywords'])

In [ ]:
# Bảng total share theo ado
df_items_share_ado_grow = total_share_items(items_ado_grow_month,'ado')

In [ ]:
# Sản phẩm xu hướng tăng ado
from metrics import items_trend_grow
items_trend_grow_ado = items_trend_grow(df_items_diff= df_items_grow_diff_ado, df_items_share= df_items_share_ado_grow ,metric='ado')
items_trend_grow_ado

,level1_kpi_category,level2_kpi_category,keywords,share_max_3m,increase_score
29078,Home & Technical Supplies,Construction Materials,sơn mạ crom chrome sáng,0.002296,14.514376
9014,Home & Technical Supplies,Construction Materials,combo 1 36 sàn nhựa,0.017836,14.252545
35055,Home & Technical Supplies,Construction Materials,xe đẩy hàng 4 bánh,0.014876,9.625985
33976,Home & Technical Supplies,Construction Materials,vòi bát inox 304 bồn,0.005848,8.721595
21758,Home & Technical Supplies,Construction Materials,máy phun sơn cầm tay,0.008175,8.628116
...,...,...,...,...,...
29079,Vehicle Essentials,Vehicle Maintenance,sơn mạ crom chrome sáng,0.007844,14.514376
12980,Vehicle Essentials,Vehicle Maintenance,dung dịch đầu bò lốc,0.023798,13.619637
35825,Vehicle Essentials,Vehicle Maintenance,đa năng moto cleaner sức,0.006752,12.626984
29190,Vehicle Essentials,Vehicle Maintenance,sơn samurai nhám lót phủ,0.014850,12.380251


##Kết quả

In [ ]:
from IPython.display import Markdown, display
from metrics import table_view_grow
from metrics import table_view_down

display(Markdown(" ## XU HƯỚNG NGÀNH HÀNG"))
display(Markdown(" ### Ngành hàng có xu hướng tăng"))
display(Markdown("\n \n".join(insight_01)))
display(Markdown("\n \n".join(insight_02)))
display(Markdown(" ### Ngành hàng có xu hướng giảm"))
display(Markdown("\n \n".join(insight_03)))
display(Markdown("\n \n".join(insight_04)))

display(Markdown(" ## XU HƯỚNG SẢN PHẨM"))
display(Markdown(" ### Sản phẩm có xu hướng tăng ADO"))

display(table_view_grow(items_trend_grow_ado))

display(Markdown(" ### Sản phẩm có xu hướng giảm ADO"))

display(table_view_down(items_trend_down_ado))

 ## XU HƯỚNG NGÀNH HÀNG

 ### Ngành hàng có xu hướng tăng

**Vehicle Essentials** 


 
 - **ADO**: Ngành hàng có xu hướng tăng **Mechanical Parts** (T8 +74.85 ADO, T9 +240.15 ADO), **Mobility Accessories** (T8 +32.97 ADO, T9 +56.94 ADO), **Personal Mobility** (T8 +26.84 ADO, T9 +9.61 ADO), **Safety Gear** (T8 +696.89 ADO, T9 +410.23 ADO), **Two-Wheel Vehicles** (T8 +0.16 ADO, T9 +0.02 ADO).
 
 - **GMV**: Ngành hàng có xu hướng tăng **Mechanical Parts** (T8 +522.06 GMV, T9 +1166.66 GMV), **Personal Mobility** (T8 +1663.79 GMV, T9 +5077.19 GMV), **Safety Gear** (T8 +3572.13 GMV, T9 +1375.56 GMV), **Vehicle Add-ons** (T8 +15.57 GMV, T9 +2088.19 GMV).

**Home & Technical Supplies** 


 
 - **ADO**: Ngành hàng có xu hướng tăng **Construction Materials** (T8 +38.68 ADO, T9 +290.34 ADO).
 
 - **GMV**: Ngành hàng có xu hướng tăng **Construction Materials** (T8 +548.22 GMV, T9 +3716.92 GMV), **Electrical Components** (T8 +508.24 GMV, T9 +1050.87 GMV), **Heavy-duty Equipment** (T8 +1626.14 GMV, T9 +1376.46 GMV), **Manual Tools** (T8 +253.21 GMV, T9 +3672.84 GMV).

 ### Ngành hàng có xu hướng giảm

...

 ## XU HƯỚNG SẢN PHẨM

 ### Sản phẩm có xu hướng tăng ADO

,Level 1,Level 2,Sản phẩm tiêu biểu| ΔADO lũy kế
0,Home & Technical Supplies,Construction Materials,"sơn mạ crom chrome sáng (+15), combo 1 36 sàn nhựa (+14), xe đẩy hàng 4 bánh (+10), vòi bát inox 304 bồn (+9), máy phun sơn cầm tay (+9)"
1,Home & Technical Supplies,Electrical Components,"công tắc bập bênh kcd1 (+30), mạch sạc pin tp4056 cổng (+20), bộ tua vít đa năng (+19), ổ cắm điện đa năng (+19), ổ cắm điện usb đa (+17)"
2,Home & Technical Supplies,Heavy-duty Equipment,"công tắc bập bênh kcd1 (+30), bộ tua vít đa năng (+19), thân máy mài chân pin (+18), thiếc hàn 63a 63 đường (+15), đỡ máy cắt cầm tay (+12)"
3,Home & Technical Supplies,Manual Tools,"cờ lê đa năng vặn (+27), bộ tua vít đa năng (+19), nắp chặn lỗ cống thoát (+15), máy siết bulong pin workfix (+14), thân máy siết bulong hitachi (+14)"
4,Home & Technical Supplies,Safety Solutions,"khoá dây cáp chống cắt (+14), khóa ổ chống cắt circle (+13), khóa đồng 3p 4p 5p (+5), két sắt hình quyển sách (+5), chốt khóa inox 90 180 (+4)"
5,Home & Technical Supplies,Support Supplies,"bộ sạc 4 pin aa (+86), keo dính đa năng mạnh (+66), pin cmos 2032 3v panasonic (+41), pin sạc lithium 18650 6800mah (+32), thước gofuther 3m 5m 7 (+29)"
6,Vehicle Essentials,In-car Utilities,"bộ mô hình 6 con (+21), rèm nắng ô tô chắn (+17), ổ cắm nguồn đầu tẩu (+16), bọc vô lăng ô tô (+14), thảm lót cốp xe ô (+13)"
7,Vehicle Essentials,Maintenance Fluids,"hòa dyramid xăng con vệ (+43), nhớt fuchs silkolene pro 4 (+8), dung dịch vệ sinh buồng (+8), nhớt xe máy tora 20w50 (+6), nhớt tổng hợp kixx ultra (+5)"
8,Vehicle Essentials,Mechanical Parts,"đèn trợ sáng bi cầu (+191), xi nhan chữ z trung (+131), 01 dây đèn led mí (+51), sạc ắc quy ô tô (+30), đèn led pha bi cầu (+30)"
9,Vehicle Essentials,Mobility Accessories,"đèn trợ sáng xe máy (+55), bơm xe đạp áp suất (+34), đèn xe đạp thể thao (+25), bi sắt xe đạp 6 (+21), bộ tua vít đa năng (+19)"


 ### Sản phẩm có xu hướng giảm ADO

,Level 1,Level 2,Sản phẩm tiêu biểu| ΔADO lũy kế
0,Home & Technical Supplies,Construction Materials,"xe kéo đi chợ thị (-8), lưới nắng 90 may đóng (-7), đầu vòi bồn chén tăng (-6), bạt nắng mưa trời đa (-6), xe kéo đi thị chợ (-4)"
1,Home & Technical Supplies,Electrical Components,"tụ quạt 400v các 1uf (-9), ống silicon dẻo chịu nhiệt (-8), công tắc hẹn giờ thông (-6), bộ công tắc hẹn giờ (-6), bộ điều khiển công tắc (-4)"
2,Home & Technical Supplies,Heavy-duty Equipment,"ê tô kẹp bàn vật (-11), dây xe 10m 15 20 (-9), thiếc hàn nhựa thông sn60 (-8), ống xoắn pu jisan hàn (-8), đầu bơm máy xe áp (-7)"
3,Home & Technical Supplies,Manual Tools,"ống silicon dẻo chịu nhiệt (-8), máy khoan pin đầu kẹp (-6), bộ 3 mũi khoan bước (-6), búa cao su đa năng (-6), búa đa năng cầm tay (-5)"
4,Home & Technical Supplies,Safety Solutions,"combo 2 cháy 4kg cứu (-3), chốt tự đóng inox 304 (-1), két sắt điện tử xiaomi (-1), combo hai cháy có đồng (-1), cháy bột abc 4kg cứu (-1)"
5,Home & Technical Supplies,Support Supplies,"keo đa năng dính nhựa (-14), keo bọt nở xốp foam (-10), keo nến chất lượng cao (-10), 40 toshiba aaa gắn remote (-7), keo vải uhu nhanh khô (-6)"
6,Vehicle Essentials,In-car Utilities,"rèm nắng theo xe hút (-7), sáp thơm areon car247 ken (-5), thảm taplo da cacbon ô (-4), đệm ô tô nệm xe (-4), rèm nắng xe ô tô (-3)"
7,Vehicle Essentials,Maintenance Fluids,"dầu nhớt xe tay ga (-11), vệ sinh sên xích bảo (-9), dầu nhớt xe số motul (-4), dung dịch mát động cơ (-4), combo nhớt máy láp shell (-4)"
8,Vehicle Essentials,Mechanical Parts,"dây led audi chạy hiệu (-5), đèn led cảnh báo nguy (-3), bộ 2 kẹp ắc quy (-2), cáp đấu trực tiếp điện (-2), bộ vá lốp không săm (-2)"
9,Vehicle Essentials,Mobility Accessories,"yên xe đạp thể thao (-9), 700 bi 7mm chống rỉ (-8), bánh xe đạp trẻ em (-4), đổi nguồn từ 36v 72v (-3), yên xe đạp active chống (-3)"


Note: Làm tương tự cho xu hướng sản phẩm với metric gmv


